In [1]:
%matplotlib inline

import os
import sys
import shutil
import copy
import time
import pickle
# import multiprocessing as mp
import numpy as np
import matplotlib.pyplot as plt
from tqdm import trange

# from lib.vrp_helper_legacy import random_vrptw
from lib.vrp_helper_0 import random_vrptw
from lib.vrp_helper_0 import check_vrp_route_validity, check_vrp_routes_validity
from lib.vrp_helper_1 import get_distMat, get_inbalance_distMat, get_solDist
from lib.vrp_helper_1 import parse_vrp_question, parse_vrp_answer, getQADict

import os.path
import subprocess
from subprocess import STDOUT,PIPE

############################################################
# 与 Java 交换数据相关的函数

def write_data_file(data, path="data.txt"):
    lines = []
    for i in range(data.shape[0]):
        lines.append(" ".join([str(x) for x in data[i]])+"\n")
    with open(path, "w+") as f:
        for line in lines:
            f.write(line)

def compile_java(java_file):
    subprocess.check_call(['javac', java_file])

def execute_java(java_file, stdin):
    java_class,ext = os.path.splitext(java_file)
    cmd = ['java', java_class]+stdin
    proc = subprocess.Popen(cmd, stdin=PIPE, stdout=PIPE, stderr=STDOUT)
    stdout,stderr = proc.communicate("".encode())
    return stdout.decode()

def parse_output(_output):
    dist, routes = _output.split(":")
    dist = float(dist)
    routes = [[int(y) for y in x.split(",")] for x in routes.split(";")]
    return dist, routes

def vrp_java(java_file, stdin):
    
    def compile_java(java_file):
        subprocess.check_call(['javac', java_file])
    
    def execute_java(java_file, stdin):
        java_class,ext = os.path.splitext(java_file)
        cmd = ['java', java_class]+stdin
        proc = subprocess.Popen(cmd, stdin=PIPE, stdout=PIPE, stderr=STDOUT)
        stdout,stderr = proc.communicate("".encode())
        return stdout.decode()
    
    def parse_output(_output):
        dist, routes = _output.split(":")
        dist = float(dist)
        routes = [[int(y) for y in x.split(",")] for x in routes.split(";")]
        return dist, routes
    
    compile_java(java_file)
    _output = execute_java(java_file, stdin)
    return parse_output(_output)

In [ ]:
n_iter = 1e6
start_time = time.time()
cap, data = parse_vrp_question("instances/x-n101-k25.txt")
# cap, data = parse_vrp_question("data/solomon_100/rc208.txt")
data = data[:51]
write_data_file(data, path="data.txt")
args = [os.path.abspath(os.getcwd())+"/data.txt", str(cap), str(int(n_iter))]
dist, routes = vrp_java('java_solverSISR.java', args)
print(time.time()-start_time, dist, routes)
check_vrp_routes_validity(cap, data, routes)\